In [1]:
import numpy as np 
import pandas as pd 
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/spam-mails-dataset/spam_ham_dataset.csv


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
from wordcloud import WordCloud
import spacy

nlp = spacy.load("en_core_web_sm")

In [3]:
df=pd.read_csv(r"/kaggle/input/spam-mails-dataset/spam_ham_dataset.csv")
df.head()

,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,2349,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,3624,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1
4,2030,ham,Subject: re : indian springs\r\nthis deal is t...,0


In [5]:
email_subjects=[]
email_text=[]
def split_subject(text):
    global email_subjects
    global email_text
    subject=""
    for ch in text:
        if(ch=='\r'):
            break
        subject+=ch
        
    email_subjects.append(subject)
    email_text.append(text.replace(subject,""))

In [6]:
df["text"]=df["text"].str.replace("Subject:","")
df.loc[df["label"]=="ham","label"]=0
df.loc[df["label"]=="spam","label"]=1
df["text"].apply(split_subject)
df["subject"]=email_subjects
df["text"]=email_text
df.head()

,Unnamed: 0,label,text,label_num,subject
0,605,0,\r\nthis is a follow up to the note i gave you...,0,enron methanol ; meter # : 988291
1,2349,0,\r\n( see attached file : hplnol 09 . xls )\r\...,0,"hpl nom for january 9 , 2001"
2,3624,0,"\r\nho ho ho , we ' re around to that most won...",0,neon retreat
3,4685,1,\r\nabasements darer prudently fortuitous unde...,1,"photoshop , windows , office . cheap . main t..."
4,2030,0,\r\nthis deal is to book the teco pvr revenue ...,0,re : indian springs


In [7]:
def swap_columns(df, col1, col2):
    col_list = list(df.columns)
    x, y = col_list.index(col1), col_list.index(col2)
    col_list[y], col_list[x] = col_list[x], col_list[y]
    df = df[col_list]
    return df
df=df.drop(df.columns[0], axis=1)
df=df.drop('label_num', axis=1)
df=swap_columns(df, 'label', 'subject')
df

,subject,text,label
0,enron methanol ; meter # : 988291,\r\nthis is a follow up to the note i gave you...,0
1,"hpl nom for january 9 , 2001",\r\n( see attached file : hplnol 09 . xls )\r\...,0
2,neon retreat,"\r\nho ho ho , we ' re around to that most won...",0
3,"photoshop , windows , office . cheap . main t...",\r\nabasements darer prudently fortuitous unde...,1
4,re : indian springs,\r\nthis deal is to book the teco pvr revenue ...,0
...,...,...,...
5166,put the 10 on the ft,\r\nthe transport volumes decreased from 25000...,0
5167,3 / 4 / 2000 and following noms,\r\nhpl can ' t take the extra 15 mmcf / d ove...,0
5168,calpine daily gas nomination,"\r\n>\r\n>\r\njulie , as i mention earlier we ...",0
5169,industrial worksheets for august 2000 activity,\r\nattached are the worksheets for august 200...,0


In [15]:
import string
import re

def clean_text(s): 
    for cs in s:
        if  not cs in string.ascii_letters:
            s = s.replace(cs, ' ')
    return s.rstrip('\r\n')

def remove_little(s): 
    wordsList = s.split()
    k_length=2
    resultList = [element for element in wordsList if len(element) > k_length]
    resultString = ' '.join(resultList)
    return resultString

def lemmatize_text(text):
    doc = nlp(text)
    lemmatized_text = " ".join([token.lemma_ for token in doc])
    return lemmatized_text

df['text'] = df['text'].apply(lambda x: clean_text(x))
df['text'] = df['text'].apply(lambda x: remove_little(x))
df['text'] = df['text'].apply(lambda x: lemmatize_text(x))

df['subject'] = df['subject'].apply(lambda x: clean_text(x))
df['subject'] = df['subject'].apply(lambda x: remove_little(x))
df['subject'] = df['subject'].apply(lambda x: lemmatize_text(x))

df

,subject,text,label
0,enron methanol meter,this follow the note give you monday prelimina...,0
1,hpl nom for january,see attach file hplnol xls hplnol xls,0
2,neon retreat,around that most wonderful time the year neon ...,0
3,photoshop windows office cheap main trending,abasement darer prudently fortuitous undergone...,1
4,indian spring,this deal book the teco pvr revenue understand...,0
...,...,...,...
5166,put the the,the transport volume decrease from all should ...,0
5167,and follow nom,hpl can take the extra mmcf over the weekend t...,0
5168,calpine daily gas nomination,julie mention early hope start the unit this a...,0
5169,industrial worksheet for august activity,attach the worksheet for august activity there...,0


In [17]:
from wordcloud import STOPWORDS 
more_stopwords = {'re' , 's' , 'subject','hpl','hou','enron'}
STOPWORDS = STOPWORDS.union(more_stopwords)

In [18]:
# Save the DataFrame to a CSV file
output_file_path = 'processed_dataset.csv'  # Change the file name/path as needed
df.to_csv(output_file_path, index=False)

print(f"Dataset saved to {output_file_path}")


Dataset saved to processed_dataset.csv


In [19]:
df

,subject,text,label
0,enron methanol meter,this follow the note give you monday prelimina...,0
1,hpl nom for january,see attach file hplnol xls hplnol xls,0
2,neon retreat,around that most wonderful time the year neon ...,0
3,photoshop windows office cheap main trending,abasement darer prudently fortuitous undergone...,1
4,indian spring,this deal book the teco pvr revenue understand...,0
...,...,...,...
5166,put the the,the transport volume decrease from all should ...,0
5167,and follow nom,hpl can take the extra mmcf over the weekend t...,0
5168,calpine daily gas nomination,julie mention early hope start the unit this a...,0
5169,industrial worksheet for august activity,attach the worksheet for august activity there...,0
